In [ ]:
# 전처리 동안 사용할 라이브러리 종합
import pandas as pd
import numpy as np

import re
import copy
import datetime
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
sns.set(font="Malgun Gothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

import warnings
warnings.filterwarnings('ignore')

## 전처리

### 공시자료 전처리

In [ ]:
company_info = pd.read_csv("../data/raw_data/all_ent.csv", encoding='cp949')
company_info = company_info[company_info["회사명"] == "삼성전자"]
company_info.reset_index(drop=True, inplace=True)

In [ ]:
company_info.columns

In [ ]:
use_cols = [
    '보고일자',
    '변동일',
    '회사명',
    '보고자 명칭',
    '종목코드',
    '보고구분',
    '보고자 구분',
    '임원',
    '직위명',
    '주요주주',
    '변동사유',
    '특정증권등의 종류',
    '변동전',
    '증감',
    '변동후',
    '발행주식 총수']
company_info = company_info[use_cols]

In [ ]:
company_info.head()

In [ ]:
# 종목코드 정리
company_info["종목코드"] = company_info["종목코드"].apply(lambda x: str(x).zfill(6))

# 변동일 양식 정리
company_info["변동일"] = company_info["변동일"].apply(lambda x: re.sub(r"[^0-9]","",x))

# str을 datetime 형식으로 변경
company_info.loc[:, "보고일자":"변동일"] = company_info.loc[:, "보고일자":"변동일"].applymap(lambda x: datetime.datetime.strptime(str(x),"%Y%m%d"))

# 일단은 당장 안쓸꺼므로 제거. 나중에 쓸수도 있음.
company_info.drop(columns="변동일", inplace=True)

# 보고자명칭 가운데 공백 제거
company_info["보고자 명칭"] = company_info["보고자 명칭"].apply(lambda x: x.replace(" ",""))

# 각 공시를 그룹화하기 위해 인덱스 생성
company_info["인덱스"] = company_info[["보고일자","보고자 명칭","보고구분"]].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)
company_info.drop(columns="보고자 명칭", inplace=True)

# 데이터프레임 컬럼순서 변경
col0=company_info.columns[-1:].to_list()
col1=company_info.columns[:-1].to_list()
new_col=col0+col1
company_info = company_info[new_col]
new_col

# 인덱스별로 정렬
company_info.sort_values("인덱스", ascending=False, ignore_index=True, inplace=True)

# 인덱스별로 groupby하면서 sum
temp = company_info[["인덱스","변동전","증감","변동후"]].groupby("인덱스").sum("증감")
temp.reset_index(inplace=True)
temp.sort_values("인덱스", ascending=False, ignore_index=True, inplace=True)

company_info.drop_duplicates(subset="인덱스", ignore_index=True, inplace=True)
company_info.drop(columns=["변동전", "증감", "변동후"], inplace=True)
company_info = pd.merge(company_info, temp, how='left', left_on='인덱스', right_on='인덱스')
company_info.set_index("인덱스", inplace=True)

company_info.head()

In [ ]:
company_info.info()

In [ ]:
# 저장.
file_name = "../data/preprocessed_data/samsung.csv"
company_info.to_csv(file_name, encoding='cp949')